In [1]:
import os
import numpy as np 
import pandas as pd
import seaborn
import seaborn as sns
import shap # for SHAP value
import random
import warnings
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
from bs4 import BeautifulSoup
import re

# The Natural Language Toolkit, or more commonly NLTK, is a suite of libraries and programs for symbolic and 
# statistical natural language processing for English written in the Python programming language.
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


#TQDM is a progress bar library with good support for nested loops and Jupyter/IPython notebooks.
from tqdm import tqdm


# Use Keras Tensorflow deeplearning library

from tensorflow import set_random_seed

from keras.utils import to_categorical
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense,Dropout,Embedding,LSTM
from keras.callbacks import EarlyStopping
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.models import Sequential


from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz

from sklearn.metrics import roc_curve,auc,make_scorer, accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix



import eli5 # for permutation importance
from eli5.sklearn import PermutationImportance

from pdpbox import pdp, info_plots # flor partial plots 

from scipy import interp

from itertools import cycle

from joblib import dump, load
seed_value = 123
os.environ['PYTHONHASHSEED']=str(seed_value)

Using TensorFlow backend.


In [2]:
os.getcwd()
warnings.filterwarnings("ignore", category=UserWarning, module='bs4')

In [3]:
set_random_seed(123)
random.seed(123)

In [4]:
np.random.seed(123)
pd.options.mode.chained_assignment = None  #hide any pandas warnings
%matplotlib inline

In [5]:
train= pd.read_csv("input/train.csv")
test = pd.read_csv("input/test.csv")

In [6]:
# Train dataset: Need only class as "Sentiment" and text as 'Phrase'
train = train.rename(columns={'class':'Sentiment','tweet':'Phrase'})
train = train.drop(['Unnamed: 0', 'count', 'hate_speech', 'offensive_language', 'neither'],axis=1).copy()

In [7]:
# Test dataset: Need only text as 'Phrase'
test = test.rename(columns={'Text':'Phrase'})
test = test.drop(['Date', 'Favorites', 'Retweets', 'Tweet ID'],axis=1).copy()


In [8]:
# # Need to Save the split texts before cleaning and tokenizing
# # Collect dependent values and convert to ONE-HOT encoding
# # Output using to_categorical
# target_t = train.Sentiment.values
# y_target_t = to_categorical(target_t)

# # Save the texts before tokenizing (must use the same random seed)
# X_train_t, X_val_t, y_train_t, y_val_t = train_test_split(train,y_target_t,
#                                                           test_size=0.2,
#                                                           random_state=seed_value,
#                                                           stratify=y_target_t)

In [9]:
def clean_sentences(df):
    tweets = []
    
#     for sent in tqdm(df['Phrase']):
    for sent in df['Phrase']:
        # remove non-alphabetic characters
        tweet_text = re.sub("[^a-zA-Z]"," ", str(sent))
        
        #remove html content
        tweet_text = BeautifulSoup(tweet_text).get_text()
        
        # tokenize
        words = word_tokenize(tweet_text.lower())
        
        # lemmatize each word to its lemma
        lemma_words = [lemmatizer.lemmatize(i) for i in words]
        
        tweets.append(lemma_words)
        
    return(tweets)

In [10]:
# cleaned tweets for both train and test set retrieved

train_sentences = clean_sentences(train)

test_sentences = clean_sentences(test)

In [11]:
# Collect dependent values and convert to ONE-HOT encoding
# Output using to_categorical
target = train.Sentiment.values
y_target = to_categorical(target)
num_classes = y_target.shape[1]

In [12]:
# Split into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(train_sentences,
                                                  y_target,
                                                  test_size=0.2,
                                                  random_state=seed_value,
                                                  stratify=y_target)

In [13]:
# Getting the no of unique words and max length of a tweet available in the list of cleaned tweets
# It is needed for initializing tokenizer of keras and subsequent padding

# Build an unordered collection of unique elements.
unique_words = set()
len_max = 0

# for sent in tqdm(X_train):
for sent in X_train:

    unique_words.update(sent)
    
    if(len_max<len(sent)):
        len_max=len(sent)

# length of the list of unique_words gives the number of unique words

print(len(list(unique_words)))
print(len_max)

28804
53


In [14]:
# Actual tokenizer of keras and convert to sequences

tokenizer = Tokenizer(num_words=len(list(unique_words)))
tokenizer.fit_on_texts(list(X_train))

# texts_to_sequences
# ARGUMENTS: list of texts to turn to sequences
# RETURN: list of sequences (one per text input)

X_train = tokenizer.texts_to_sequences(X_train)
X_val = tokenizer.texts_to_sequences(X_val)
X_test = tokenizer.texts_to_sequences(test_sentences)

In [15]:
# Padding is done to equalize the lengths of all input tweets.
# LTSM networks need all inputs to be same length.
# Therefore, tweets lesser than max length will be made equal using extra zeros at end. This is padding.
# Also, you always have to give a three-dimensional array as an input to your LSTM network

X_train = sequence.pad_sequences(X_train, maxlen=len_max)
X_val = sequence.pad_sequences(X_val, maxlen=len_max)
X_test = sequence.pad_sequences(X_test, maxlen=len_max)

In [16]:
# Early stopping to prevent overfitting deep learning neural network models
# This is a method that allows you to specify an arbitrary large number of training epochs.
# This stops training once the model performance stops improving on a hold out validation dataset


early_stopping = EarlyStopping(min_delta = 0.001, mode = 'max', monitor = 'val_acc', patience = 2)
callback = [early_stopping]

In [17]:
# re-run model everytime new parameter changes (must run sequential to initialize epoch)
# Model with just acc as metric

model = Sequential()
model.add(Embedding(len(list(unique_words)),300,input_length=len_max))
model.add(LSTM(128,dropout=0.5,recurrent_dropout=0.5,return_sequences=True))
model.add(LSTM(64,dropout=0.5,recurrent_dropout=0.5,return_sequences=False))
model.add(Dense(100,activation='relu')) #try elu
model.add(Dropout(0.5))
model.add(Dense(num_classes,activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['acc'])
model.summary()

W0803 18:16:04.904451 4642133440 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0803 18:16:04.907301 4642133440 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0803 18:16:04.909843 4642133440 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0803 18:16:04.987577 4642133440 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0803 18:16:04.994526 4642133440 deprecati

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 53, 300)           8641200   
_________________________________________________________________
lstm_1 (LSTM)                (None, 53, 128)           219648    
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               6500      
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 303       
Total params: 8,917,059
Trainable params: 8,917,059
Non-trainable params: 0
_________________________________________________________________


In [18]:
# re-run model everytime new parameter changes (must run sequential to initialize epoch)
# Model with no metrics

model_1 = Sequential()
model_1.add(Embedding(len(list(unique_words)),300,input_length=len_max))
model_1.add(LSTM(128,dropout=0.5,recurrent_dropout=0.5,return_sequences=True))
model_1.add(LSTM(64,dropout=0.5,recurrent_dropout=0.5,return_sequences=False))
model_1.add(Dense(100,activation='relu')) #try elu
model_1.add(Dropout(0.5))
model_1.add(Dense(num_classes,activation='softmax'))
model_1.compile(optimizer='adam',
              loss='categorical_crossentropy')
model_1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 53, 300)           8641200   
_________________________________________________________________
lstm_3 (LSTM)                (None, 53, 128)           219648    
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               6500      
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 303       
Total params: 8,917,059
Trainable params: 8,917,059
Non-trainable params: 0
_________________________________________________________________


In [19]:
# create pandas df for y_train
# Describe classifier hate (0), offensive (1), neither or neutral (2)

y_df = pd.DataFrame(y_train)
y_df.sum()

0     1144.0
1    15348.0
2     3330.0
dtype: float32

In [20]:
# Count of number of data points in each category
hate_ct = y_train.sum(axis = 0)[0]
offensive_ct = y_train.sum(axis = 0)[1]
neither_ct = y_train.sum(axis = 0)[2]
total_ct = y_train.sum()

In [21]:
# Calculating the inverse ratio of each category to use for the weights of the model
inv_ratio_hate = 1 - (hate_ct / total_ct)
inv_ratio_hurtful = 1 - (offensive_ct / total_ct)
inv_ratio_neither = 1 - (neither_ct / total_ct)

In [22]:
# Sum up the no. of records for each classifier
hate_num = y_df[0].sum()
offensive_num = y_df[1].sum()
neutral_num = y_df[2].sum()

In [23]:
# Do inverse ratio so that hate and neutral has a higher weight
print(f' K Hate:{1 - hate_num / len(y_df)}')
print(f' S Hate:{1 - (hate_num / len(y_df))}')

print(f'K neutral: {1 - neutral_num / len(y_df)}')
print(f'S neutral: {1 - (neutral_num / len(y_df))}')

# Inverse ratio will give offensive a lower weight
print(f'K Offensive: {1 - offensive_num / len(y_df)}')
print(f'S Offensive: {1 - (offensive_num / len(y_df))}')

 K Hate:0.9422863485016648
 S Hate:0.9422863485016648
K neutral: 0.8320048431036222
S neutral: 0.8320048431036222
K Offensive: 0.22570880839471297
S Offensive: 0.22570880839471297


In [25]:
# fit the model adjusting for epochs, batch, and weight

model.fit(
    X_train, y_train, 
    validation_data=(X_val,y_val),
    epochs=15, #may not run all due to callback
    batch_size=256, #faster with larger batch_size but it's generalizing
    verbose=1,
    callbacks=callback, #stops training once the model stops improving. Prevents overfitting.
    class_weight={0: inv_ratio_hate,
                  1: inv_ratio_hurtful,
                  2: inv_ratio_neither} #use inverse ratio to set hate with highest weight (somewhat arbitrary)
) 

Train on 19822 samples, validate on 4956 samples
Epoch 1/15
19822/19822 [==============================] - 42s 2ms/step - loss: 0.1370 - acc: 0.9025 - val_loss: 0.3304 - val_acc: 0.8816
Epoch 2/15
19822/19822 [==============================] - 45s 2ms/step - loss: 0.0935 - acc: 0.9215 - val_loss: 0.3651 - val_acc: 0.8745
Epoch 3/15
19822/19822 [==============================] - 44s 2ms/step - loss: 0.0651 - acc: 0.9439 - val_loss: 0.4268 - val_acc: 0.8553


In [26]:
model_1.fit(
    X_train, y_train, 
    validation_data=(X_val,y_val),
    epochs=15, #may not run all due to callback
    batch_size=256, #faster with larger batch_size but it's generalizing
    verbose=1,
    callbacks=callback, #stops training once the model stops improving. Prevents overfitting.
    class_weight={0: inv_ratio_hate,
                  1: inv_ratio_hurtful,
                  2: inv_ratio_neither} #use inverse ratio to set hate with highest weight (somewhat arbitrary)
) 

Train on 19822 samples, validate on 4956 samples
Epoch 1/15
19822/19822 [==============================] - 47s 2ms/step - loss: 0.3204 - val_loss: 0.4069
Epoch 2/15
19822/19822 [==============================] - 57s 3ms/step - loss: 0.1732 - val_loss: 0.3392
Epoch 3/15
19822/19822 [==============================] - 53s 3ms/step - loss: 0.1135 - val_loss: 0.3615
Epoch 4/15
19822/19822 [==============================] - 51s 3ms/step - loss: 0.0807 - val_loss: 0.3872
Epoch 5/15
19822/19822 [==============================] - 61s 3ms/step - loss: 0.0568 - val_loss: 0.4540
Epoch 6/15
19822/19822 [==============================] - 129s 6ms/step - loss: 0.0434 - val_loss: 0.4688
Epoch 7/15
19822/19822 [==============================] - 66s 3ms/step - loss: 0.0330 - val_loss: 0.5105
Epoch 8/15
19822/19822 [==============================] - 39s 2ms/step - loss: 0.0254 - val_loss: 0.5859
Epoch 9/15
19822/19822 [==============================] - 39s 2ms/step - loss: 0.0203 - val_loss: 0.5695
Epoch

In [27]:
# Predict validation sentiment!

y_pred = model.predict(X_val)
# print(y_pred)

y_pred_1 = model_1.predict(X_val)

In [28]:
# Keras and Sklearn read arrays differently
# Create function to convert keras array to show only one highest sentiment result per list
def keras_output_sklearn(y):
    
    result = []
    
    for element in y:
        result.append(np.argmax(element))
        

    return result

In [29]:
print(classification_report(keras_output_sklearn(y_val), keras_output_sklearn(y_pred)))

              precision    recall  f1-score   support

           0       0.30      0.50      0.38       286
           1       0.93      0.89      0.91      3838
           2       0.83      0.79      0.81       832

   micro avg       0.86      0.86      0.86      4956
   macro avg       0.69      0.73      0.70      4956
weighted avg       0.88      0.86      0.87      4956



In [30]:
print(classification_report(keras_output_sklearn(y_val), keras_output_sklearn(y_pred_1)))

              precision    recall  f1-score   support

           0       0.35      0.35      0.35       286
           1       0.92      0.92      0.92      3838
           2       0.81      0.82      0.81       832

   micro avg       0.87      0.87      0.87      4956
   macro avg       0.70      0.70      0.70      4956
weighted avg       0.87      0.87      0.87      4956



In [ ]:
# # Report for model with 'acc' metric
# report = classification_report(keras_output_sklearn(y_val), keras_output_sklearn(y_pred))
# print(report)

In [ ]:
# Report for model with no metrics 
report_1 = classification_report(keras_output_sklearn(y_val), keras_output_sklearn(y_pred_1))
print(report_1)

In [ ]:
# Saving model with 'acc' metric
model.save('../models/model_acc.h5')

# Saving model with no metrics
model_1.save('../models/model_no.h5')